#### For this homework you can get 100 points + 20 bonus points. The bonus points will be counted into your total homework score until you get the maximum homework score, 400.

Copying and pasting other people's code is absolutely prohibited.  I will report to the education team if I find any such cases. Collaboration and discussion is highly encouraged, and feel free to exchange ideas with your classmates, but write your own code please. 

### Question 1: Accuracy and interpretability (10 pts)

a) Describe a real-world prediction problem using urban data for which _interpretability_ of your models and results is essential, and for which it might be preferable to use decision trees rather than random forests.  Argue why this is the case. (3 pts)





### For example, if we want to use building energy consumption data to predict how many energy a building can use, it will be essential for the customer to understand why this building can consume this amount of energy instead of a simple answer. Using decision trees rather than random forests can provide a more interpretable model to the customer. 

b) Describe a real-world prediction problem using urban data for which accuracy is paramount and interpretability may be less important, and for which it might be preferable to use random forests rather than decision trees. Argue why this is the case. (3 pts)


### If we want to use census data and health record to predict the occurrence of a disease, accuracy will be our priority in this case. Using random forests can provide a better result, in this case, is more preferable. 

c) Let's imagine that you want to try to get the best of both worlds (accuracy _and_ interpretability).  So you decide to start by learning a random forest classifier.  Describe at least one way of getting some interpretability out of the model by post-processing.  You could either pick a method from the literature (e.g., Domingos's work on combining multiple models or some method of computing variable importance), or come up with your own approach (doesn't have to be ground-breaking, but feel free to be creative!) (4 pts)

**From a paper I found about how to Make Tree Ensembles Interpretable, the paper mentioned Post-hoc ATM Interpretation Method, which is:
1.reducing the number of regions, while 2. minimizing model error**

**I think it is a pretty good way to decrease the complexity of the model meanwhile minimizing the error. That way, we will have a model that is easy to understand and still have a rather good proformance.**

Ref:https://arxiv.org/pdf/1606.05390.pdf



###  Question 2: Build a tree by hand following exactly the lecture notes. Note that the dataset has been slightly modified, so you will get a different tree than the one shown in the lecture notes. (30 pts + 20 pts)

30 points for parts a, b, c, d, f.
20 bonus points for optional part e.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data='MPG, cylinders, HP, weight\ngood, 4, 75, light\nbad, 6, 90, medium\nbad, 4, 110, medium\nbad, 8, 175, weighty\nbad, 6, 95, medium\nbad, 4, 94, light\nbad, 4, 95, light\nbad, 8, 139, weighty\nbad, 8, 190, weighty\nbad, 8, 145, weighty\nbad, 6, 100, medium\ngood, 4, 92, medium\nbad, 6, 100, weighty\nbad, 8, 170, weighty\ngood, 4, 89, medium\ngood, 4, 65, light\nbad, 6, 85, medium\ngood, 4, 81, light\nbad, 6, 95, medium\ngood, 4, 93, light'

In [3]:
data=",".join(data.split(", "))
print data

MPG,cylinders,HP,weight
good,4,75,light
bad,6,90,medium
bad,4,110,medium
bad,8,175,weighty
bad,6,95,medium
bad,4,94,light
bad,4,95,light
bad,8,139,weighty
bad,8,190,weighty
bad,8,145,weighty
bad,6,100,medium
good,4,92,medium
bad,6,100,weighty
bad,8,170,weighty
good,4,89,medium
good,4,65,light
bad,6,85,medium
good,4,81,light
bad,6,95,medium
good,4,93,light


#### Please use numpy and pandas to do the calculation for parts a) through d):



**_a) Prepare the data set to a pandas dataframe from the given string (2 pts)_**


In [4]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

In [5]:
data_1 = StringIO(data)
df = pd.read_csv(data_1, sep=",")
df.head(5)

,MPG,cylinders,HP,weight
0,good,4,75,light
1,bad,6,90,medium
2,bad,4,110,medium
3,bad,8,175,weighty
4,bad,6,95,medium


**_b) Start with the entire dataset and find the most common value (3 pts)_**

In [6]:
#most common value in each column
df.mode()
#(axis = 0)

,MPG,cylinders,HP,weight
0,bad,4,95,medium


In [7]:
#most common value in the whole dataset
items_counts = df['MPG'].value_counts()
items_counts.append(df['cylinders'].value_counts())
items_counts.append(df['HP'].value_counts())
items_counts.append(df['weight'].value_counts())
items_counts.idxmax()


'bad'

#### So MPG will be our target value.

**_c) Use "information gain" as your decision rule to split your data into two groups. What is the split rule and what is the maximum value of the information gain? (5 pts)_**

In [8]:
def RealValue (column, target_column):
    DecisionRule_h = []
    for i in range(len(column) - 1):
        if target_column[i] != target_column[i+1]:
            DecisionRule_h.append((float(column[i]) + float(column[i+1]))/2)
    return DecisionRule_h

In [9]:
#Split rule: Discrete: choose a class, split into = and !=
DecisionRule_c = df.cylinders.unique()
DecisionRule_w = df.weight.unique()

In [10]:
# Real: choose a threshold, first sort the value and use the midpoints.
df = df.sort('HP')
df = df.reset_index(drop = True)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [11]:
DecisionRule_h = RealValue(df.HP, df.MPG)

In [12]:
DecisionRule_h

[83.0, 87.0, 89.5, 91.0, 93.5]

In [13]:
def Rule(DecisionRule_c,DecisionRule_w,DecisionRule_h):
    DecisionRule = []
    for x in map(lambda x: x, DecisionRule_c):
        DecisionRule.append('df.cylinders == {}'.format(x))
    
    for x in map(lambda x: x, DecisionRule_w):
        DecisionRule.append('df.weight == \'{}\''.format(x))
    
    for x in map(lambda x: x, DecisionRule_h):
        DecisionRule.append('df.HP > {}'.format(x))
    
    rule = pd.DataFrame({'Split': DecisionRule})
    rule['Gain'] = np.zeros(len(DecisionRule))
    return rule

In [14]:
rule = Rule(DecisionRule_c,DecisionRule_w,DecisionRule_h)
rule

,Split,Gain
0,df.cylinders == 4,0.0
1,df.cylinders == 6,0.0
2,df.cylinders == 8,0.0
3,df.weight == 'light',0.0
4,df.weight == 'medium',0.0
5,df.weight == 'weighty',0.0
6,df.HP > 83.0,0.0
7,df.HP > 87.0,0.0
8,df.HP > 89.5,0.0
9,df.HP > 91.0,0.0


In [15]:
def Gain(Y_G, Y_B, N_G, N_B):
    def F(x, y):
        if x == 0:
            F = y * np.log2((float(x) + float(y)) / float(y))
        elif y == 0:
            F = x * np.log2((float(x) + float(y)) / float(x))
        else:
            F = x * np.log2((float(x) + float(y)) / float(x)) + y * np.log2((float(x) + float(y)) / float(y))
        return F
    G = (F((Y_G + N_G), (Y_B + N_B)) - F(Y_G, Y_B) - F(N_G, N_B))/(Y_G + Y_B + N_G + N_B)
    return G

In [16]:
def InformationGain(Rule, df):
    for i in range(len(Rule)):
        condition_s = eval(Rule.Split[i])
        #condition_t = df.MPG == 'good'
        data_y = df[condition_s]
        data_n = df[~condition_s]
        y_g = len(data_y[df.MPG == 'good'])
        y_b = len(data_y[~(df.MPG == 'good')])
        n_g = len(data_n[df.MPG == 'good'])
        n_b = len(data_n[~(df.MPG == 'good')])
        g = Gain(y_g, y_b, n_g, n_b)
        Rule['Gain'].iloc[i] = g
        

In [17]:
InformationGain(rule, df)
rule

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Split,Gain
0,df.cylinders == 4,0.468058
1,df.cylinders == 6,0.191631
2,df.cylinders == 8,0.153078
3,df.weight == 'light',0.191631
4,df.weight == 'medium',0.005802
5,df.weight == 'weighty',0.191631
6,df.HP > 83.0,0.309840
7,df.HP > 87.0,0.162065
8,df.HP > 89.5,0.275927
9,df.HP > 91.0,0.191631


In [18]:
GainMax = rule.Gain.max()
MaxRule = rule[rule.Gain == GainMax]
MaxRule

,Split,Gain
10,df.HP > 93.5,0.55678


** d) Repeat the process b) and c) until that you can perfectly split the training data.  Show the resulting decision tree in a format of your choice, as long as the tree structure and the prediction at each leaf node are clearly shown.  Note that you are _not_ expected to prune the tree in parts d) and e). (10 pts) **



In [19]:
def BuildDecisionTree (df):
    node = []
    while (len(df) != 0): 
        DecisionRule_c = df.cylinders.unique()
        DecisionRule_w = df.weight.unique()
        df = df.sort('HP')
        df = df.reset_index(drop = True)
        DecisionRule_h = RealValue(df.HP, df.MPG)
        rule = Rule(DecisionRule_c,DecisionRule_w,DecisionRule_h)
        InformationGain(rule, df)
        GainMax = rule.Gain.max()
        MaxRule = rule[rule.Gain == GainMax]
        r = MaxRule.index[0]
        condition = eval(rule.Split[r])
        node.append(rule.Split[r])
        data_y = df[condition]
        data_n = df[~condition]
        y_g = len(data_y[df.MPG == 'good'])
        y_b = len(data_y[~(df.MPG == 'good')])
        n_g = len(data_n[df.MPG == 'good'])
        n_b = len(data_n[~(df.MPG == 'good')])
        if(y_g != 0 and y_b != 0):
            df = data_y
            
        elif(n_g != 0 and n_b != 0):
            if len(df) == len(data_y):
                df.append(data_n)
            else:
                df = data_n
        elif((n_g == 0 or n_b != 0) and (y_g == 0 or y_b == 0)):
            df = []
    return node

In [20]:
nodes = BuildDecisionTree(df)
nodes

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/pyt

['df.HP > 93.5', 'df.cylinders == 4']

In [21]:
def printtree(nodes, df):
    for i in range(len(nodes)):
        if i < (len(nodes)-1):
            print ('{}.'.format(i) + nodes[i] + '?')
            condition = eval(nodes[i])
            data_y = df[condition]
            data_n = df[~condition]
            y_g = len(data_y[df.MPG == 'good'])
            y_b = len(data_y[~(df.MPG == 'good')])
            n_g = len(data_n[df.MPG == 'good'])
            n_b = len(data_n[~(df.MPG == 'good')])
            if(y_g != 0 and y_b != 0):
                df = data_y
                if n_g == 0:
                    print ('F-> BAD: {}'.format(n_b))
                    print ('T->')
                elif n_b == 0:
                    print ('F-> GOOD: {}'.format(n_g))
                    print ('T->')
            elif(n_g != 0 and n_b != 0):
                df = data_n
                if y_g == 0:
                    print ('T-> BAD: {}'.format(y_b))
                    print ('F->')
                elif y_b == 0:
                    print ('T-> GOOD: {}'.format(y_g))
                    print ('F->')            
        else:
            print ('{}.'.format(i) + nodes[i] + '?')
            condition = eval(nodes[i])
            data_y = df[condition]
            data_n = df[~condition]
            y_g = len(data_y[df.MPG == 'good'])
            y_b = len(data_y[~(df.MPG == 'good')])
            n_g = len(data_n[df.MPG == 'good'])
            n_b = len(data_n[~(df.MPG == 'good')])
            if n_g == 0:
                print ('F-> BAD: {}'.format(n_b))
                if y_g == 0:
                    print ('T-> BAD: {}'.format(y_b))
                elif y_b == 0:
                    print ('T-> GOOD: {}'.format(y_g))
            if n_b == 0:
                print ('F-> GOOD: {}'.format(n_g))
                if y_g == 0:
                    print ('T-> BAD: {}'.format(y_b))
                elif y_b == 0:
                    print ('T-> GOOD: {}'.format(y_g))
            

In [22]:
printtree(nodes, df)

0.df.HP > 93.5?
T-> BAD: 12
F->
1.df.cylinders == 4?
F-> BAD: 2
T-> GOOD: 6


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:33: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:35: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/

e)*OPTIONAL- 20 bonus points* 
Define a function: Tree(data_train, data_test) which learns a decision tree from data_train and uses it to predict the values for data_test.
Example:

##### Input of the desired function:

data_train="good,4,75,light\nbad,6,90,medium\nbad,4,110,medium"

data_test="?,6,95,medium\n?,4,93,light"

##### Output of the desired function should be data_test with the unknown values replaced by your tree's predictions, e.g.:

data_test_predicted="bad,6,95,medium\ngood,4,93,light"




In [66]:
def Tree(data_train, data_test):
    nodes = BuildDecisionTree(data_train)
    for i in range(len(nodes)):
        if i < (len(nodes)-1):
            condition = eval(nodes[i])
            data_y = data_train[condition]
            data_n = data_train[~condition]
            y_g = len(data_y[data_train.MPG == 'good'])
            y_b = len(data_y[~(data_train.MPG == 'good')])
            n_g = len(data_n[data_train.MPG == 'good'])
            n_b = len(data_n[~(data_train.MPG == 'good')])
            if(y_g != 0 and y_b != 0):
                data_train = data_y
                if n_g == 0:
                    data_test.MPG[~condition] = 'bad'
                elif n_b == 0:
                    data_test.MPG[~condition] = 'good'
                data_test = data_test[condition]
            elif(n_g != 0 and n_b != 0):
                data_train = data_n
                if y_g == 0:
                    data_test.MPG[condition] = 'bad'
                elif y_b == 0:
                    data_test.MPG[condition] = 'good'
                data_test = data_test[~condition]
        else:
            condition = eval(nodes[i])
            data_y = data_train[condition]
            data_n = data_train[~condition]
            y_g = len(data_y[data_train.MPG == 'good'])
            y_b = len(data_y[~(data_train.MPG == 'good')])
            n_g = len(data_n[data_train.MPG == 'good'])
            n_b = len(data_n[~(data_train.MPG == 'good')])
            if n_g == 0:
                data_test.MPG[~condition] = 'bad'
                if y_g == 0:
                    data_test.MPG[condition] = 'bad'
                elif y_b == 0:
                    data_test.MPG[condition] = 'good'
            if n_b == 0:
                data_test.MPG[~condition] = 'good'
                if y_g == 0:
                    data_test.MPG[condition] = 'bad'
                elif y_b == 0:
                    data_test.MPG[condition] = 'good'
    return data_test

In [67]:
data_test = "MPG, cylinders, HP, weight\n?,6,95,medium\n?,4,93,light"
data_test=",".join(data_test.split(", "))
data_test = StringIO(data_test)
df_test = pd.read_csv(data_test, sep=",")
df_test.head(5)

,MPG,cylinders,HP,weight
0,?,6,95,medium
1,?,4,93,light


In [69]:
df_test = Tree(df,df_test)
df_test

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/pyt

,MPG,cylinders,HP,weight
0,good,6,95,medium
1,good,4,93,light


**f) Classify the following five vehicles as having "good" or "bad" fuel efficiency (miles per gallon).  You can do this by hand using the tree structure learned in part d), or automatically using the function you wrote in part e). (10 pts)**

?,4,93,weighty
?,8,70,light
?,6,113,medium
?,6,95,weighty
?,4,115,medium




In [70]:
data_test = "MPG, cylinders, HP, weight\n?,4,93,weighty\n?,8,70,light\n?,6,113,medium\n?,6,95,weighty\n?,4,115,medium"
data_test=",".join(data_test.split(", "))
data_test = StringIO(data_test)
df_test = pd.read_csv(data_test, sep=",")
df_test.head(5)


,MPG,cylinders,HP,weight
0,?,4,93,weighty
1,?,8,70,light
2,?,6,113,medium
3,?,6,95,weighty
4,?,4,115,medium


In [71]:
df_test = Tree(df,df_test)
df_test

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/pyt

,MPG,cylinders,HP,weight
0,good,4,93,weighty
1,good,8,70,light
2,good,6,113,medium
3,bad,6,95,weighty
4,good,4,115,medium


### Question 3, Predicting burden of disease （40 pts)

In [ ]:
data=pd.read_csv("https://serv.cusp.nyu.edu/classes/ML_2016_Spring/ML_2017/Burden of diarrheal illness by country.csv")
print("Here are the first three rows:")
data.iloc[0:3,:]

#### Your goal is to train a decision tree classifier for the attribute “BurdenOfDisease" using all other variables (except country name) as features using sklearn.tree.DecisionTreeClassifier. http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

a) Please choose a test/train split and choose a hyper-parameter governing model simplicity. For example, the maximum tree depth or maximum number of leaf nodes. Then, fit your decision tree classifier for different values of this parameter and for each such value, record the corresponding AUC score. (10 pts)

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.metrics import roc_auc_score as rs
import matplotlib.pylab as plt
%matplotlib inline
from sklearn.externals.six import StringIO  
from sklearn import tree
from IPython.display import Image  
import pydotplus

In [ ]:
lb = preprocessing.LabelBinarizer()
lb.fit(data.BurdenOfDisease)
lb.classes_

In [ ]:
data.BurdenOfDisease = lb.transform(data.BurdenOfDisease)


In [ ]:
# Prepare target variable and feature space:
Y=data.loc[:,"BurdenOfDisease"]

#Get the features space and make dummy variables. 
X=data.loc[:,"FrxnPeaceIn10":"FemaleLtrcyRate"]
X=pd.get_dummies(X)

#a)Split data set to training and testing data:

X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.2, random_state=999)

In [ ]:
#hyper-parameter: the Max Depth
real=np.array(Y_test.apply(int))
AUC=[]
for i in range(1,10,2):    
    rf = DTC(max_depth=i)
    rf.fit(X_train, Y_train)
    pred=rf.predict_proba(X_test)[:,1]
    AUC.append(rs(real,pred))

In [ ]:
plt.figure(figsize=(7,5))
plt.plot(range(1,10,2),AUC)
plt.xlabel("Max Depth")
plt.ylabel("AUC")
plt.title("AUC vs Simplicity (max depth)")
#plt.axvline(32,color='r',linestyle='--')
plt.xlim(2,10)
plt.show()

In [ ]:
# Max-depth = 3
AUC_IS=[]
AUC_OS=[]
for i in range(10):
    X_train,X_test,Y_train,Y_test=train_test_split(X, Y, test_size=0.2)
    rf=DTC(max_depth=3)
    rf.fit(X_train,Y_train)
    pred=rf.predict(X_test)
    AUC_OS.append(rs(np.array(Y_test.apply(int)),pred))
    AUC_IS.append(rs(np.array(Y_train.apply(int)),rf.predict(X_train)))

In [ ]:
print "For Max Depth equals to 3:"
print "IS AUC: {}".format(np.mean(AUC_IS))
print "OS AUC: {}".format(np.mean(AUC_OS))

In [ ]:
# Max-depth = 5
AUC_IS=[]
AUC_OS=[]
for i in range(10):
    X_train,X_test,Y_train,Y_test=train_test_split(X, Y, test_size=0.3)
    rf=DTC(max_depth=5)
    rf.fit(X_train,Y_train)
    pred=rf.predict(X_test)
    AUC_OS.append(rs(np.array(Y_test.apply(int)),pred))
    AUC_IS.append(rs(np.array(Y_train.apply(int)),rf.predict(X_train)))

In [ ]:
print "For Max Depth equals to 5:"
print "IS AUC: {}".format(np.mean(AUC_IS))
print "OS AUC: {}".format(np.mean(AUC_OS))

In [ ]:
# Max-depth = 8
AUC_IS=[]
AUC_OS=[]
for i in range(10):
    X_train,X_test,Y_train,Y_test=train_test_split(X, Y, test_size=0.2)
    rf=DTC(max_depth=8)
    rf.fit(X_train,Y_train)
    pred=rf.predict(X_test)
    AUC_OS.append(rs(np.array(Y_test.apply(int)),pred))
    AUC_IS.append(rs(np.array(Y_train.apply(int)),rf.predict(X_train)))

In [ ]:
print "For Max Depth equals to 8:"
print "IS AUC: {}".format(np.mean(AUC_IS))
print "OS AUC: {}".format(np.mean(AUC_OS))

b) Make a plot of performance vs. simplicity for different values of the hyper-parameter chosen in part a). That is, the x-axis should be hyper-parameter value (e.g. tree depth) and the y-axis should be AUC score. (10 pts)

In [ ]:
#hyper-parameter: the Max Depth
real=np.array(Y_test.apply(int))
AUC=[]
for i in range(1,10,2):    
    rf = DTC(max_depth=i)
    rf.fit(X_train, Y_train)
    pred=rf.predict_proba(X_test)[:,1]
    AUC.append(rs(real,pred))

In [ ]:
plt.figure(figsize=(7,5))
plt.plot(range(1,10,2),AUC)
plt.xlabel("Max Depth")
plt.ylabel("AUC")
plt.title("AUC vs Simplicity (max depth)")
plt.axvline(3,color='r',linestyle='--')
plt.axvline(5,color='r',linestyle='--')
plt.axvline(8,color='r',linestyle='--')
plt.xlim(2,10)
plt.show()


c) Tune the hyper-parameter you choose in part a) by cross-validation using the training data. You can choose to use package from sklearn or write your own code to do cross-validation by spliting the training data into training and validation data. What is the OS accuracy after tuning the hyper-parameter? (10 pts)

In [ ]:
#max_depth=8
OS=[]
for i in range(10):
    X_train,X_test,Y_train,Y_test=train_test_split(X, Y, test_size=0.2)
    rf=DTC(max_depth=8)
    rf.fit(X_train,Y_train)
    OS.append(rf.score(X_test,Y_test))

In [ ]:
print ('Average OS Accuracy for 8 max depth: {}'.format(np.mean(OS)))

d) Visualize a simple decision tree (e.g. a “shallow” tree, or a tree with
few leaf nodes) classifier and report its performance. You can draw
the decision tree by hand or use a graphical representation (e.g.
http://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html), but make sure it is easy to understand (e.g. the
features chosen for each split should be clearly labeled in each
internal node, as well as the prediction at each leaf node). (10 pts)

In [ ]:
Feature_importance=pd.DataFrame([list(X_train.columns),list(rf.feature_importances_)]).T
Feature_importance.columns=["variables","importance"]
Feature_importance.sort_values(by="importance",ascending=False).iloc[:3,:]

In [ ]:
X_train_simple=X_train.loc[:,["SustAccImprSanUrb","SustAccImprWatUrb","SustAccImprSanRur"]]
X_test_simple=X_test.loc[:,["SustAccImprSanUrb","SustAccImprWatUrb","SustAccImprWatRur"]]

In [ ]:
real=np.array(Y_test.apply(int))
rf = DTC(max_depth=4) # Here of course we could remove this limit. But you could have a very 
                             #big graph for next question.
rf.fit(X_train_simple, Y_train)
pred=rf.predict_proba(X_test_simple)[:,1]
print("The AUC score for this simple model with 3 features is : {}".format(rs(real,pred)))

In [ ]:
dot_data = StringIO()  
tree.export_graphviz(rf, out_file=dot_data,  
                         feature_names=["SustAccImprSanUrb","SustAccImprWatUrb","SustAccImprWatRur"],  
                         class_names=["awful", "high", "low", "medium"], 
                     filled=True, rounded=True, special_characters = True)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())  

### Question 4, Fit a random forest to the data from question 3 (20 pts)

a) Please use the same test/train split from previous question and feel free to tune the hyper-parameters for Random Forest model using training data. The package from sklearn is here: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html.
Then please report your out of sample prediction result and compare this model's performance with 2c). (10 pts)



In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC


In [ ]:
rf = RFC(n_estimators=30, n_jobs=-1,max_depth = 8)
rf.fit(X_train, Y_train)

In [ ]:
pred=rf.predict_proba(X_test)[:,1]
print "OS AUC: {}".format(rs(np.array(Y_test.apply(int)),pred))

In [ ]:
#Practice Three. Let's fix max_depth = 8, try to build trees from 1 to 30 
#and take a look the AUC. (n_estimators from range(1,30,2))
res=[]
for i in range(1,30,2):
    rf = RFC(n_estimators=i, n_jobs=4,max_depth = 8)
    rf.fit(X_train, Y_train)
    pred=rf.predict_proba(X_test)[:,1]
    res.append(rs(np.array(Y_test.apply(int)),pred))
print "OS AUC: {}".format(np.mean(res))

b) Write one paragraph comparing the results from those two models (Random Forest vs Decision Tree) in terms of both accuracy and interpretability. (10 pts)

The average AUC score for Decision Tree from Question 3 is 0.819230769231 for 8 max depth, and 0.956140350877 for Random Forest with the same max depth, which indicate that random forest model has a better accuracy prefomance(17% better). But by using decision tree, we can have a clear plot which can show how the model works for individual data. Random forest, however, since it is a ensemble model, it's really hard to either understand or interprete how it works. 